**Link to the streamlit App**

https://mlassignment3-error404.streamlit.app/


In [15]:
%pip install pyarrow
%pip install huggingface-hub
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from nltk.lm import Vocabulary
from nltk.tokenize import sent_tokenize, word_tokenize
# import nltk
# nltk.download('punkt_tab')
import torch
import torch.nn.functional as F
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pprint import pprint
import numpy as np
import torch._dynamo
torch._dynamo.config.suppress_errors = True
import re
import time
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [17]:
try:
  !pip install torch_xla
  import torch_xla
  import torch_xla.core.xla_model as xm
  device = xm.xla_device()
  print("TPU is available. Using TPU")
except:
  if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
  else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")

print(device)

CUDA is available. Using GPU.
cuda


We will train eight models, with two block sizes, two embedding dimensions, and two activation functions.

In [18]:
block_sizes = [16, 32]
emb_dims = [128, 256]
layer_sizes = [512, 512, 512]

Our dataset consists of Middle school level maths questions, which is provided online free of cost by microsoft.

In [19]:
df = pd.read_parquet("hf://datasets/microsoft/orca-math-word-problems-200k/data/train-00000-of-00001.parquet")

We will tokenize these symbols separately, i.e. if "10%2" is present in the text, it will be converted to "1 0 % 2". The word_tokenize function basically splits using the blankspace and removes the empty tokens and newline characters.

In [20]:
symbols = r"([+\-/*^=0123456789.,|'()])"

In [21]:
def tokenize(text):
    text = text.replace("\n", " NEWLINE_CHAR ")
    text = re.sub(symbols, r" \1 ", text)
    tokens = word_tokenize(text)
    tokens = ["\n" if token == "NEWLINE_CHAR" else token for token in tokens]
    tokens.append('\n')
    return tokens

We make a list of lists of questions tokenized. We also save the unique tokens in the tokenset.

In [22]:
str_arr = []
tokenset=set()
for i in range(len(df)):
    token_arr = tokenize(df.iloc[i, 0])
    str_arr.append(token_arr)
    tokenset.update(token_arr)
    if i%10000 == 0:
        print(i)

0


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000


We add an extra token "END" to signify the filler token, and make a dictionary mapping from tokens to integers.

In [23]:
all_words = ['END']+sorted(list(tokenset))
stoi = {s:i for i, s in enumerate(all_words)}
itos = {i:s for s, i in stoi.items()}
int_text_arr = [[stoi[i] for i in str_arr[j]] for j in range(len(str_arr))]
print(len(all_words))

26605


We define a NextWord model class, using which we make pytorch model objects.

In [24]:
class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size, activation):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin_1 = nn.Linear(block_size * emb_dim, hidden_size[0])
        self.lin_2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.lin_3 = nn.Linear(hidden_size[1], hidden_size[2])
        # self.lin_1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin_4 = nn.Linear(hidden_size[2], vocab_size)
        self.activation = activation

    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = self.activation(self.lin_1(x))
        x = self.activation(self.lin_2(x))
        x = self.activation(self.lin_3(x))
        x = self.lin_4(x)
        return x

The code below makes (and trains) a model with a specified context (block) size, embedding dimension, hidden layer sizes, and the activation. We will be using three hidden layers of 512, 512, 512 nodes each for every model of ours. We will vary the embedding dimensions, the context window, and the activation function applied to every node. We have a constant learning rate of 0.001, use the Adam optimizer and train it for 20 epochs.

Our training dataset consists of 1,20,00,000 samples. (Which is the total number of words in the dataset)

In [25]:
def makemodel(model_name, block_size, emb_dim, hidden_size, activation, path="", lr=0.001, epochs = 20):
  X = []
  Y = []
  for i in range(len(int_text_arr)):
      context =[0]*block_size
      for j in range(len(int_text_arr[i])):
          Y.append(int_text_arr[i][j])
          X.append(context)
          context = context[1:] + [int_text_arr[i][j]]
      X.append(context)
      Y.append(0)
      if i%10000 == 0:
          print(i)
  X = torch.tensor(X).to(device)
  Y = torch.tensor(Y).to(device)
  torch.manual_seed(42)
  perm = torch.randperm(X.shape[0])
  X = X[perm]
  Y = Y[perm]
  print(len(X))
  def load_model(path = ""):
    if path == "":
        model = NextWord(block_size, len(stoi), emb_dim, hidden_size, activation).to(device)
        model = torch.compile(model)
        return model
    model = NextWord(block_size, len(stoi), emb_dim, hidden_size, activation).to(device)
    state_dict = torch.load(path)
    state_dict = {k.replace("_orig_mod.", ""): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    return model
  torch.manual_seed(42)
  emb = torch.nn.Embedding(len(stoi), emb_dim)
  model = load_model(path = path)
  print("Compiled")
  torch.save(model.state_dict(), f'{model_name}.pth')
  loss_fn = nn.CrossEntropyLoss()
  opt = torch.optim.AdamW(model.parameters(), lr=lr)
  # Mini-batch training
  batch_size = 4096
  print_every = 1
  elapsed_time = []
  for epoch in range(epochs):
      start_time = time.time()
      for i in range(0, X.shape[0], batch_size):
          x = X[i:i+batch_size]
          y = Y[i:i+batch_size]
          y_pred = model(x)
          loss = loss_fn(y_pred, y)
          loss.backward()
          opt.step()
          opt.zero_grad()
          print(f"Model: {model_name} {int(i/batch_size)} Minibatch out of {int(len(X)/batch_size)} done, {epoch} done, loss: {loss.item()}")
      end_time = time.time()
      elapsed_time.append(end_time - start_time)
      print(f"Model: {model_name} Epoch {epoch} done, elapsed time: {end_time - start_time}")
      if epoch % print_every == 0:
          print(epoch, loss.item())
          torch.save(model.state_dict(), f'{model_name}.pth')
  torch.save(model.state_dict(), f'{model_name}.pth')

We now train all models using the dataset we have made, and save them.

In [26]:
hidden_size = [512, 512, 512]
# EMB: 128, block_size: 16, Act: tanH, Arch: 512, 512, 512
for activation in [nn.LeakyReLU(), nn.Tanh()]:
  for j in [128,256]:
    for k in [16,32]:
      if(k == 16 and activation == nn.LeakyReLU): continue
      makemodel(f'{activation}_EMB{j}_Context{k}', block_size = k, emb_dim = j, hidden_size = [512,512,512], activation = activation)